In [13]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import transforms, models
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
from PIL import Image
import time
from pathlib import Path
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# データセットのパスを設定
data_dir = 'SDNET2018'
train_dir = {
    'D': {
        'crack': os.path.join(data_dir, 'D', 'CD'),
        'uncrack': os.path.join(data_dir, 'D', 'UD')
    },
    'P': {
        'crack': os.path.join(data_dir, 'P', 'CP'),
        'uncrack': os.path.join(data_dir, 'P', 'UP')
    },
    'W': {
        'crack': os.path.join(data_dir, 'W', 'CW'),
        'uncrack': os.path.join(data_dir, 'W', 'UW')
    }
}

# 結果保存用のディレクトリ作成
os.makedirs('results', exist_ok=True)
os.makedirs('models', exist_ok=True)

# データ変換の定義
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomRotation(10),
        transforms.ColorJitter(brightness=0.2, contrast=0.2),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
}

class CrackDataset(Dataset):
    def __init__(self, crack_dir, uncrack_dir, transform=None):
        self.transform = transform
        self.image_paths = []
        self.labels = []
        
        # ひび割れ画像の追加
        for img_name in os.listdir(crack_dir):
            if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                self.image_paths.append(os.path.join(crack_dir, img_name))
                self.labels.append(1)
        
        # 非ひび割れ画像の追加
        for img_name in os.listdir(uncrack_dir):
            if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                self.image_paths.append(os.path.join(uncrack_dir, img_name))
                self.labels.append(0)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        try:
            img_path = self.image_paths[idx]
            image = Image.open(img_path).convert('RGB')
            label = self.labels[idx]
            
            if self.transform:
                image = self.transform(image)
            
            return image, label
        except Exception as e:
            print(f"Error loading image {img_path}: {str(e)}")
            return self.__getitem__((idx + 1) % len(self))

def train_model(model, dataloaders, criterion, optimizer, scheduler, device, num_epochs=10):
    print(f'Starting training on {device}...')
    start_time = time.time()
    best_model_wts = model.state_dict()
    best_acc = 0.0
    
    history = {
        'train_loss': [], 'train_acc': [],
        'val_loss': [], 'val_acc': []
    }
    
    for epoch in range(num_epochs):
        print(f'\nEpoch {epoch+1}/{num_epochs}')
        
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
            
            running_loss = 0.0
            running_corrects = 0
            total_samples = 0
            
            with tqdm(dataloaders[phase], desc=f'{phase.capitalize()} Phase') as pbar:
                for inputs, labels in pbar:
                    try:
                        inputs = inputs.to(device)
                        labels = labels.to(device)
                        
                        optimizer.zero_grad()
                        
                        with torch.set_grad_enabled(phase == 'train'):
                            outputs = model(inputs)
                            _, preds = torch.max(outputs, 1)
                            loss = criterion(outputs, labels)
                            
                            if phase == 'train':
                                loss.backward()
                                optimizer.step()
                        
                        batch_loss = loss.item() * inputs.size(0)
                        # CPUに移動してからnumpy/itemを使用
                        batch_corrects = torch.sum(preds == labels.data).cpu().item()
                        running_loss += batch_loss
                        running_corrects += batch_corrects
                        total_samples += inputs.size(0)
                        
                        current_loss = batch_loss / inputs.size(0)
                        current_acc = (batch_corrects / inputs.size(0)) * 100
                        pbar.set_postfix({
                            'loss': f'{current_loss:.4f}',
                            'acc': f'{current_acc:.2f}'
                        })
                    except Exception as e:
                        print(f"Error in batch: {str(e)}")
                        continue
            
            epoch_loss = running_loss / total_samples
            epoch_acc = (running_corrects / total_samples) * 100
            
            if phase == 'train':
                scheduler.step()
                history['train_loss'].append(epoch_loss)
                history['train_acc'].append(epoch_acc)
            else:
                history['val_loss'].append(epoch_loss)
                history['val_acc'].append(epoch_acc)
            
            print(f'{phase.capitalize()} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.2f}%')
            
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()
                # ベストモデルの保存
                torch.save(best_model_wts, os.path.join('models', f'best_model_temp.pth'))
    
    time_elapsed = time.time() - start_time
    print(f'\nTraining completed in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:.2f}%')
    
    # 最後にベストなモデルの重みをロード
    model.load_state_dict(best_model_wts)
    return model, history

def save_results(model, history, val_loader, surface_type, device):
    # 学習履歴のプロット
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 2, 1)
    plt.plot(history['train_loss'], label='Training Loss')
    plt.plot(history['val_loss'], label='Validation Loss')
    plt.title(f'Loss History ({surface_type})')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(history['train_acc'], label='Training Accuracy')
    plt.plot(history['val_acc'], label='Validation Accuracy')
    plt.title(f'Accuracy History ({surface_type})')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig(os.path.join('results', f'training_history_{surface_type}.png'))
    plt.close()
    
    # モデルの評価
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for inputs, labels in tqdm(val_loader, desc='Evaluating'):
            try:
                inputs = inputs.to(device)
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.numpy())
            except Exception as e:
                print(f"Error in evaluation batch: {str(e)}")
                continue
    
    # 混同行列の生成と保存
    try:
        cm = confusion_matrix(all_labels, all_preds)
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                    xticklabels=['No Crack', 'Crack'],
                    yticklabels=['No Crack', 'Crack'])
        plt.title(f'Confusion Matrix ({surface_type})')
        plt.ylabel('True Label')
        plt.xlabel('Predicted Label')
        plt.savefig(os.path.join('results', f'confusion_matrix_{surface_type}.png'))
        plt.close()
        
        # 分類レポートの保存
        report = classification_report(all_labels, all_preds, 
                                     target_names=['No Crack', 'Crack'])
        with open(os.path.join('results', f'classification_report_{surface_type}.txt'), 'w') as f:
            f.write(report)
    except Exception as e:
        print(f"Error in generating evaluation metrics: {str(e)}")
    
    # モデルの保存
    try:
        # ベストモデルをコピー
        os.rename(
            os.path.join('models', 'best_model_temp.pth'),
            os.path.join('models', f'best_model_{surface_type}.pth')
        )
    except Exception as e:
        print(f"Error saving model: {str(e)}")
        # 通常のモデル保存を試みる
        torch.save(model.state_dict(), os.path.join('models', f'best_model_{surface_type}.pth'))

def train_surface_type(surface_type, device):
    print(f"\nTraining model for surface type: {surface_type}")
    
    try:
        # データセットの作成
        full_dataset = CrackDataset(
            train_dir[surface_type]['crack'],
            train_dir[surface_type]['uncrack'],
            transform=data_transforms['train']
        )
        
        # データセット情報の表示
        print(f"Total images: {len(full_dataset)}")
        crack_count = sum(1 for label in full_dataset.labels if label == 1)
        no_crack_count = sum(1 for label in full_dataset.labels if label == 0)
        print(f"Crack images: {crack_count}")
        print(f"No crack images: {no_crack_count}")
        
        # データセットの分割
        train_size = int(0.8 * len(full_dataset))
        val_size = len(full_dataset) - train_size
        train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])
        
        # DataLoaderの作成
        train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=0)
        val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=0)
        
        dataloaders = {
            'train': train_loader,
            'val': val_loader
        }
        
        # モデルの設定
        model = models.resnet50(pretrained=True)
        num_ftrs = model.fc.in_features
        model.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(num_ftrs, 2)
        )
        model = model.to(device)
        
        # 損失関数とオプティマイザの設定
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=0.0003)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)
        
        # モデルの訓練
        model, history = train_model(
            model, dataloaders, criterion, optimizer, scheduler, device, num_epochs=2
        )
        
        # 結果の保存
        save_results(model, history, val_loader, surface_type, device)
        
        return model
        
    except Exception as e:
        print(f"Error training surface type {surface_type}: {str(e)}")
        return None

def main():
    # GPU/MPS/CPUの設定
    if torch.cuda.is_available():
        device = torch.device("cuda")
        print("Using CUDA")
    elif torch.backends.mps.is_available():
        device = torch.device("mps")
        print("Using MPS")
    else:
        device = torch.device("cpu")
        print("Using CPU")
    
    print(f"PyTorch version: {torch.__version__}")
    print(f"Device: {device}")
    
    # データディレクトリの確認
    if not os.path.exists(data_dir):
        raise FileNotFoundError(f"Data directory '{data_dir}' not found!")
    
    # トレーニングの実行
    for surface_type in ['D', 'P', 'W']:
        try:
            model = train_surface_type(surface_type, device)
            if model is not None:
                print(f"Successfully trained model for {surface_type}")
            else:
                print(f"Failed to train model for {surface_type}")
        except Exception as e:
            print(f"Fatal error training {surface_type}: {str(e)}")
            continue

if __name__ == '__main__':
    main()

Using MPS
PyTorch version: 2.5.1
Device: mps

Training model for surface type: D
Total images: 13620
Crack images: 2025
No crack images: 11595
Starting training on mps...

Epoch 1/2


Train Phase: 100%|██████████| 681/681 [10:26<00:00,  1.09it/s, loss=0.4013, acc=87.50] 


Train Loss: 0.3409 Acc: 88.12%


Val Phase: 100%|██████████| 171/171 [00:50<00:00,  3.38it/s, loss=0.1705, acc=100.00]


Val Loss: 0.3118 Acc: 89.57%

Epoch 2/2


...

Train Loss: 0.3014 Acc: 88.87%


Val Phase: 100%|██████████| 227/227 [00:52<00:00,  4.34it/s, loss=0.0666, acc=100.00]


Val Loss: 0.2809 Acc: 90.24%

Training completed in 25m 41s
Best val Acc: 90.24%


Evaluating: 100%|██████████| 227/227 [00:51<00:00,  4.39it/s]


Successfully trained model for W
